In [1]:
import torch
import torchvision
import torchvision.transforms as transforms

In [2]:
torch.cuda.set_device(0)

In [3]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

In [4]:
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4, shuffle=True, num_workers=2)
testset = torchvision.datasets.CIFAR10(root="./data", train=False, download=True, transform=transform)
testloader= torch.utils.data.DataLoader(testset, batch_size=4, shuffle=False, num_workers=2)
classes=('plane', 'car', 'bird', 'cat','deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [5]:
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # 5 Layers : 2 conv layers and 3 fully connected layers !
        self.conv1 = nn.Conv2d(3, 16, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(16, 25, 5)
        self.fc1 = nn.Linear(25 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 25 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [6]:
# For Adagrad
net = Net().cuda()
lossvsiter_adam=[]

# To see if the model is on CUDA or not !
if (next(net.parameters()).is_cuda) :
    print("The model is on CUDA")
else :
    print("The model is on CPU")

import torch.optim as optim

# Declare a loss function
criterion = nn.CrossEntropyLoss()
# Declare an optimizer
optimizer = optim.Adam(net.parameters(),lr=0.001)


# No of iterations !
iterations = 10

for epoch in range(iterations):  # loop over the dataset multiple times
    
    # Reset the loss for the current epoch !
    running_loss = 0.0
    
    # Loop over all the mini-batches therea are 12500 mini batches of size 4 each !
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs, labels = data
        
        # wrap them in Variable & if possible make them cuda tensors
        inputs, labels = Variable(inputs.cuda()), Variable(labels.cuda())

        # zero the parameter gradients for the current epoch
        optimizer.zero_grad()
        
        # forward + backward + optimize
        
        # forward
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        
        # Calculate gradients of whatever variable set to req_gardients = True
        loss.backward()
        
        # Take one step of the gradient descent for this epoch ! 
        optimizer.step()
        
        # print statistics
        running_loss += loss.data[0]
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[Epoch :: %d, Mini Batch :: %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            lossvsiter_adam.append(running_loss / 2000)
            running_loss = 0.0
        

print('Finished Training')

The model is on CUDA
[Epoch :: 1, Mini Batch ::  2000] loss: 1.802
[Epoch :: 1, Mini Batch ::  4000] loss: 1.560
[Epoch :: 1, Mini Batch ::  6000] loss: 1.487
[Epoch :: 1, Mini Batch ::  8000] loss: 1.408
[Epoch :: 1, Mini Batch :: 10000] loss: 1.380
[Epoch :: 1, Mini Batch :: 12000] loss: 1.319
[Epoch :: 2, Mini Batch ::  2000] loss: 1.236
[Epoch :: 2, Mini Batch ::  4000] loss: 1.240
[Epoch :: 2, Mini Batch ::  6000] loss: 1.224
[Epoch :: 2, Mini Batch ::  8000] loss: 1.221
[Epoch :: 2, Mini Batch :: 10000] loss: 1.220
[Epoch :: 2, Mini Batch :: 12000] loss: 1.184
[Epoch :: 3, Mini Batch ::  2000] loss: 1.105
[Epoch :: 3, Mini Batch ::  4000] loss: 1.136
[Epoch :: 3, Mini Batch ::  6000] loss: 1.105
[Epoch :: 3, Mini Batch ::  8000] loss: 1.109
[Epoch :: 3, Mini Batch :: 10000] loss: 1.119
[Epoch :: 3, Mini Batch :: 12000] loss: 1.090
[Epoch :: 4, Mini Batch ::  2000] loss: 1.028
[Epoch :: 4, Mini Batch ::  4000] loss: 1.008
[Epoch :: 4, Mini Batch ::  6000] loss: 1.056
[Epoch :: 4, 

In [7]:
import pickle
with open("lossvsiter_adam.pkl","wb") as f:
    pickle.dump(lossvsiter_adam,f)

In [8]:
correct = 0
total = 0

for data in testloader:
    images, labels = data
    images=images.cuda()
    labels=labels.cuda()
    try:
        outputs = net(Variable(images))
    except RuntimeError as re:
        print(outputs.is_cuda)
        print(str(re))
        sys.exit()
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images: 65 %


In [9]:
for i, data in enumerate(trainloader, 0):
    images, labels = data
    images=images.cuda()
    labels=labels.cuda()
    try:
        outputs = net(Variable(images))
    except RuntimeError as re:
        print(outputs.is_cuda)
        print(str(re))
        sys.exit()
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum()
    if (i%1000) == 0:
        print(i)

print('Accuracy of the network on the 50000 trained images: %d %%' % (
    100 * correct / total))

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
Accuracy of the network on the 50000 trained images: 72 %


In [10]:
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
for data in testloader:
    images, labels = data
    images=images.cuda()
    labels=labels.cuda()
    outputs = net(Variable(images))
    _, predicted = torch.max(outputs.data, 1)
    c = (predicted == labels).squeeze()
    for i in range(4):
        label = labels[i]
        class_correct[label] += c[i]
        class_total[label] += 1


for i in range(10):
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))

Accuracy of plane : 74 %
Accuracy of   car : 80 %
Accuracy of  bird : 47 %
Accuracy of   cat : 32 %
Accuracy of  deer : 62 %
Accuracy of   dog : 54 %
Accuracy of  frog : 83 %
Accuracy of horse : 66 %
Accuracy of  ship : 75 %
Accuracy of truck : 80 %
